In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import string
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from nltk import pos_tag, ne_chunk

In [ ]:
!pip install word2number
!pip install Unidecode
!pip install contractions

In [ ]:
import unidecode
import contractions
from word2number import w2n

In [ ]:
  from nltk import tag
  from nltk.tag import pos_tag
  from nltk.tree import Tree
  from nltk.chunk import ne_chunk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
def read_files(file_loc):
  '''
  This function reads tsv data from a file in the drive

  args - a string containing the files location
  returns - a list containing the text data
  '''

  dataset = []

  with open(file_loc, 'r', encoding='cp850') as file:
    for line in file:
      dataset.append(line)

  return dataset

In [ ]:
def separate_labels(dataset):
  '''This function will separate the labels/class and examples/documents from the dataset'''
  labels = []
  documents = []

  for line in dataset:
    splitted_line = line.strip().split('\t', 2)
    labels.append(splitted_line[2])
    documents.append(splitted_line[1])

  return labels, documents

In [ ]:
def separate_labels2(dataset):
  '''This function will separate the labels/class and examples/documents from the dataset'''
  labels = []
  documents = []

  for line in dataset:
    splitted_line = line.strip().split('\t', 2)
    labels.append(splitted_line[0])
    documents.append(splitted_line[1])

  return labels, documents

In [ ]:
def remove_url(documents):
  '''This function removes URL's from Texts'''
  url_removed = []

  # Your code here
  for line in documents:
    url_removed.append(re.sub('http[s]?://\S+', '', line))

  return url_removed

In [ ]:
def remove_hashtag(documents):
  '''This function will remove all occurences of # from the texts'''
  hashtag_removed = []

  # map hashtag to space
  translator = str.maketrans('#', ' '*len('#'), '')

  for line in documents:
    hashtag_removed.append(line.translate(translator))

  return hashtag_removed

In [ ]:
def remove_whitespaces(documents):
  '''This function removes multiple whitespaces and replace them with a single whitespace'''
  whitespace_removed = []

  for line in documents:
    whitespace_removed.append(' '.join(line.split()))

  return whitespace_removed

In [ ]:
def text_lowercasing(documents):
  lowercased_docs = []

  for line in documents:
    lowercased_docs.append(line.lower())
  
  return lowercased_docs

In [ ]:
# 1- Tokenization

def tokenization_data(documents):
  '''This function removes HTML's from Texts'''
  text_tokenized = []

  # Your code here
  for line in documents:
    text_tokenized.append(word_tokenize(line))

  return text_tokenized

In [ ]:
def remove_punctuation(documents):

  punct_removed = []

  for doc in documents:
    temp = []
    for word in doc:
      if word not in string.punctuation:
        temp.append(word)
    
    punct_removed.append(temp)

  return punct_removed

In [ ]:
def remove_stopwords(documents):
  
  stopword_removed = []

  stop_words = set(stopwords.words('english'))

  for doc in documents:
    stopword_removed.append([word for word in doc if word not in stop_words])
    # temp = []
    # for word in doc:
    #   if word not in stop_words:
    #     temp.append(word)
    
    # stopword_removed.append(temp)

  return stopword_removed

In [ ]:
def char_n_gram_ready(documents):
  '''This function joins the tokens in a sentence like string'''
  joined_docs = []

  for line in documents:
    joined_docs.append(' '.join(line))

  return joined_docs

In [ ]:
def vec_tfidf(tfidf = True):

  if tfidf:
    vec = TfidfVectorizer(preprocessor = identity,
                          tokenizer = identity)
  else:
  # vec = CountVectorizer(preprocessor = identity, analyzer='char',
                          # tokenizer = identity, ngram_range=(1, 3))
   vec = CountVectorizer(preprocessor = identity,
                          tokenizer = identity)
    
  return vec

In [ ]:
def SVM_Static(train_docs, train_lbls, test_docs, test_lbls):
  
  vec = vec_tfidf(tfidf=False)
    
  # combines the vectorizer with the Naive Bayes classifier
  classifier = Pipeline([('vec', vec),
                         ('cls', SVC(kernel='linear', C=1.0, gamma='scale'))])
  
  classifier.fit(train_docs, train_lbls)

  prediction = classifier.predict(test_docs)
  for lbl, doc in zip(test_lbls[:10], prediction[:10]):
    print(lbl)
    print(doc)
    print()

 

In [ ]:
def pre_processing(documents):

  documents = remove_url(documents)



  documents = text_lowercasing(documents)

  

  documents = tokenization_data(documents)



  return documents

In [ ]:
def main():
  print('Reading The Dataset...')
  
  # Reading the training data 
  training_dataset = read_files('Hate_Speech/dev.txt')
  train_labels, train_docs = separate_labels(training_dataset)
  

  # Reading the test data
  test_dataset = read_files('Hate_Speech/test.txt')
  test_labels, test_docs = separate_labels2(test_dataset)



  # calling the pre processing dunction
  train_docs = pre_processing(train_docs)
  test_docs = pre_processing(test_docs)
  # print(train_docs)


  print('\nTraining the SVM Classifier...')
  SVM_Static(train_docs, train_labels, test_docs, test_labels)
 

if __name__ == '__main__':
  main()

Reading The Dataset...

Training the SVM Classifier...
hasoc_en_902
HATE

hasoc_en_416
NONE

hasoc_en_207
OFFN

hasoc_en_595
NONE

hasoc_en_568
NONE

hasoc_en_953
NONE

hasoc_en_685
HATE

hasoc_en_672
NONE

hasoc_en_746
NONE

hasoc_en_527
NONE

